# Calculate monthly means from the 8-daily MODIS gapfilled data

Could also be used with slight modification to calculate monthly means of the original unfilled 8-daily data.



In [1]:
import numpy as np
import glob
from collections import defaultdict
import os
import bottleneck as bn
from osgeo import gdal
import rasterio
import numexpr as ne

Based on the known filename patterns generate a list of files against each year / month.

store it as a nested dictionary year : month : [files]

In [2]:
# generate this in excel with =CONCATENATE(DAYNUM,":",MONTH(DAYNUM),", ")
#daymonths = {1:1, 9:1, 17:1, 25:1, 33:2, 41:2, 49:2, 57:2, 65:3, 73:3, 81:3, 89:3, 97:4, 105:4, 113:4, 121:4, 129:5, 137:5, 145:5, 153:6, 161:6, 169:6, 177:6, 185:7, 193:7, 201:7, 209:7, 217:8, 225:8, 233:8, 241:8, 249:9, 257:9, 265:9, 273:9, 281:10, 289:10, 297:10, 305:10, 313:11, 321:11, 329:11, 337:12, 345:12, 353:12, 361:12}
daymonths = {1:1, 9:1, 17:1, 25:1, 
             33:2, 41:2, 49:2, 57:2, 
             65:3, 73:3, 81:3, 89:3, 
             97:4, 105:4, 113:4, 
             121:5, 129:5, 137:5, 145:5, 
             153:6, 161:6, 169:6, 177:6, 
             185:7, 193:7, 201:7, 209:7, 
             217:8, 225:8, 233:8, 241:8, 
             249:9, 257:9, 265:9, 273:9, 
             281:10, 289:10, 297:10, 305:10, 
             313:11, 321:11, 329:11, 
             337:12, 345:12, 353:12, 361:12}
fillDataFiles = defaultdict(lambda: defaultdict(list))
fillFlagFiles = defaultdict(lambda: defaultdict(list))

#baseDir = r'F:\MOD11A2_Night_Output\Output_Final_30k_2030pc'
#baseDir = r'E:\MCD43B4\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean'
#baseDir = r'E:\MOD11A2_DiurnalDiffs_Output\LST_Diurnal_Diffs\Output_8day'
baseDir = r'G:\Extra\Output\TCW'
baseDir = r'F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean'

genericFilePattern = r"{0}\*_{1}.tif"
#tag = "LST_DiurnalDifference"#"TCB_Filled_Data"
tag = 'Filled_Data'

for fn in glob.glob(genericFilePattern.format(baseDir,tag)):
    print fn
    datestr = os.path.basename(fn).split('_')[0][1:]
    yr = int(datestr[:4])
    day = int(datestr[4:])
    month = daymonths[day]
    fillDataFiles[yr][month].append(fn)
    fillFlagFiles[yr][month].append(fn.replace('Filled_Data','Fill_Flags'))


F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000049_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000057_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000065_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000073_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000081_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000089_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000097_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000105_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000113_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2030pc_FixedMean\A2000121_EVI_Filled_Data.tif
F:\MCD43B4_Gapfilled_Output\EVI\Output_Final_30k_2

Functions to calculate the mean and fill proportions of a given set of files.

Calculations are in memory so all the files must fit in memory at once in their entirety.

Uses the bottleneck library for more efficient calculation

Fill proportions are retrieved based on a hardcoded bitmask corresponding to that used in the MODIS gapfilling code

In [3]:
def calculateTemporalMean(inFiles):
    n = len(inFiles)
    global NDV
    with rasterio.open(inFiles[0]) as src:
        xSize = src.width
        ySize = src.height
        NDV = src.nodatavals[0]
    
    allData = np.empty(shape=(n, ySize, xSize), dtype = np.float32)
    
    for i in range(n):
        with rasterio.open(inFiles[i]) as src:
            assert src.width == xSize
            assert src.height == ySize
            assert src.nodatavals[0] == NDV
            allData[i] = src.read_band(1, masked=False)
    
    allData[allData == NDV] = np.nan
    #nanVal = np.nan
    #ne.evaluate("where(allData == NDV, nanVal, 1)",out=allData)
    meanData = bn.nanmean(allData, axis=0)
    meanData[np.isnan(meanData)] = NDV
    #ne.evaluate("where(meanData==nanVal,NDV,meanData)", out=meanData)
    
    return meanData


def calculateFillProportions(inFiles):
    # flags that indicate any type of fill (A1 partial, A1 complete, A2)
    anyFillFlag = 16 + 32 + 64
    n = len(inFiles)
    
    with rasterio.open(inFiles[0]) as src:
        xSize = src.width
        ySize = src.height
    
    allFlags = np.empty(shape=(n,ySize,xSize), dtype = np.uint8 )
    
    for i in range(n):
        with rasterio.open(inFiles[i]) as src:
            assert src.width == xSize
            assert src.height == ySize
           # assert src.nodatavals[0] == NDV
            allFlags[i] = src.read_band(1, masked=False)
        
    allFlags = np.bitwise_and(allFlags, anyFillFlag) != 0
    propFilled = bn.nanmean(allFlags, axis=0)
    
    return propFilled * 100

In [4]:
#outDir = r'F:\MOD11A2_Night_Output\Output_Monthly_Means\1km'
#outDir = r'E:\MCD43B4\MCD43B4_Gapfilled_Output\EVI\Output_Monthly_Means\1km'
#outDir = r'F:\MOD11A2_DiurnalDiffs_Output\Output_Monthly_Means\1km'
#outDir = r'H:\MCD43B4_Gapfilled_Output\TCB\Output_Monthly_Means\1km'
outDir = r'G:\Extra\Output\Aggregated\1km'
outDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MCD43B4_BRDF_Reflectance\EVI\1km\Annual'
xSize = 43200
ySize = 21600


In [5]:
aFlag = gdal.Open(fillDataFiles[2015][1][0])

In [6]:
globalGT = aFlag.GetGeoTransform()

In [7]:
globalProj = aFlag.GetProjection()

In [8]:
aFlag = None

In [ ]:

#LST_Day.2000.09.5km.Mean.tif
outputFNTemplate = r"{0!s}\{1!s}.{2!s}.{3!s}.1km.{4!s}.tif"
#metric = "LST_DiurnalDiff"#"EVI"
metric = "EVI"
#i = 0
outDrv = gdal.GetDriverByName('GTiff')
for yr, yrinfo in fillDataFiles.iteritems():
    print yr
    for monthNum, monthDataFiles in yrinfo.iteritems():
 #       assert i == 0
    
        monthFlagFiles = fillFlagFiles[yr][monthNum]
        outDataFN = outputFNTemplate.format(
            outDir, metric, yr, str(monthNum).zfill(2), "Data")
        outFlagFN = outputFNTemplate.format(
            outDir, metric, yr, str(monthNum).zfill(2), "FilledProportion")
        # create file
        
        outputDataArr = calculateTemporalMean(monthDataFiles)
        
        #templateDS = gdal.Open(monthDataFiles[0])
        outDS = outDrv.Create(outDataFN, xSize, ySize, 1, gdal.GDT_Float32,
                              ["TILED=YES","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
        outDS.SetGeoTransform(globalGT)
        outDS.SetProjection(globalProj)
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outputDataArr)
        outBand.SetNoDataValue(NDV)
        outDS = None
        #templateDS = None
        del outputDataArr
        
        outputFlagArr = calculateFillProportions(monthFlagFiles)
        
        outDS = outDrv.Create(outFlagFN, xSize, ySize, 1, gdal.GDT_Byte,
                              ["TILED=YES","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
        outDS.SetGeoTransform(globalGT)
        outDS.SetProjection(globalProj)
       
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outputFlagArr)
        outDS = None
        del outputFlagArr
        
#        i += 1

In [10]:
outputFNTemplate = r"{0!s}\{1!s}.{2!s}.{3!s}.1km.{4!s}.tif"
#metric = "LST_DiurnalDiff"#"EVI"
metric = "EVI"
#i = 0
outDrv = gdal.GetDriverByName('GTiff')
for yr, yrinfo in fillDataFiles.iteritems():
    print yr
    yrDataFiles = []
    yrFlagFiles = []
    for monthNum, monthDataFiles in yrinfo.iteritems():
 #       assert i == 0
        yrDataFiles.extend(monthDataFiles)
        yrFlagFiles.extend(fillFlagFiles[yr][monthNum])
    outDataFN = outputFNTemplate.format(
        outDir, metric, yr, "Annual", "Data")
    outFlagFN = outputFNTemplate.format(
            outDir, metric, yr, "Annual", "FilledProportion")
        # create file
        
    outputDataArr = calculateTemporalMean(yrDataFiles)
        
    #templateDS = gdal.Open(monthDataFiles[0])
    outDS = outDrv.Create(outDataFN, xSize, ySize, 1, gdal.GDT_Float32,
                              ["TILED=YES","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
    outDS.SetGeoTransform(globalGT)
    outDS.SetProjection(globalProj)
    outBand = outDS.GetRasterBand(1)
    outBand.WriteArray(outputDataArr)
    outBand.SetNoDataValue(NDV)
    outDS = None
    #templateDS = None
    del outputDataArr
        
    outputFlagArr = calculateFillProportions(yrFlagFiles)
        
    outDS = outDrv.Create(outFlagFN, xSize, ySize, 1, gdal.GDT_Byte,
                              ["TILED=YES","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
    outDS.SetGeoTransform(globalGT)
    outDS.SetProjection(globalProj)
       
    outBand = outDS.GetRasterBand(1)
    outBand.WriteArray(outputFlagArr)
    outDS = None
    del outputFlagArr
        
#        i += 1

2000


MemoryError: 